In [15]:
import pandas as pd
import numpy as np
import torch
from torch import nn
from transformers import BertModel, BertTokenizer
from tqdm import tqdm
from torch.utils.data import DataLoader, TensorDataset, Dataset

In [16]:
# df = pd.read_csv("/home/ryler/Datasets/Mcdonalds-Review-Text-Classification/McDonald_s_Reviews.csv", encoding="latin1")
df = pd.read_csv("/home/rynutty/Documents/DataSets/Mcdonalds-Reviews/McDonald_s_Reviews.csv", encoding="latin1")

In [17]:
df.head()

,reviewer_id,store_name,category,store_address,latitude,longitude,rating_count,review_time,review,rating
0,1,McDonald's,Fast food restaurant,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",3 months ago,Why does it look like someone spit on my food?...,1 star
1,2,McDonald's,Fast food restaurant,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",5 days ago,It'd McDonalds. It is what it is as far as the...,4 stars
2,3,McDonald's,Fast food restaurant,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",5 days ago,Made a mobile order got to the speaker and che...,1 star
3,4,McDonald's,Fast food restaurant,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",a month ago,My mc. Crispy chicken sandwich was ï¿½ï¿½ï¿½ï¿...,5 stars
4,5,McDonald's,Fast food restaurant,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",2 months ago,"I repeat my order 3 times in the drive thru, a...",1 star


In [18]:
df.shape

(33396, 10)

In [19]:
def clean_ratings(ratings):
    ratings = [int(rate[0]) for rate in ratings]
    cleaned_ratings = []

    for rating in ratings:
        if rating >= 4:
            cleaned_ratings.append(2)
        elif rating == 3:
            cleaned_ratings.append(1)
        else:
            cleaned_ratings.append(0)

    return cleaned_ratings

In [20]:
train_cutoff = int(len(df) * 0.8)

train = df.iloc[:train_cutoff, :]
test = df.iloc[train_cutoff:, :].reset_index(drop=True)

x_train = train["review"]
y_train = clean_ratings(train["rating"])

x_test = test["review"]
y_test = clean_ratings(test["rating"])

In [21]:


class MLP(nn.Module):

    def __init__(self, in_features, out_features):
        super().__init__()

        self.inference = nn.Sequential(
            nn.Linear(in_features, out_features),
            nn.ReLU()
        )

    def forward(self, x):
        return self.inference(x)


class Encoder(nn.Module):

    def __init__(self, d_model, num_heads, num_encoder_blocks):
        super().__init__()

        blocks = [EncoderBlock(d_model=d_model, num_heads=num_heads) for _ in range(num_encoder_blocks)]
        self.inference = nn.Sequential(*blocks)

    def forward(self, x):
        return self.inference(x)
    

class EncoderBlock(nn.Module):

    def __init__(self, d_model, num_heads):
        super().__init__()

        self.layer_norm = nn.LayerNorm(d_model)
        self.mha = nn.MultiheadAttention(embed_dim=d_model, num_heads=num_heads, dropout=0.2, batch_first=True, device="cuda")
        self.mlp = MLP(in_features=d_model, out_features=d_model)

    def forward(self, x):
        x = self.layer_norm(x)
        identity = x
        x, attention_weights = self.mha(x, x, x)
        x = x + identity
        x = self.layer_norm(x)
        identity = x
        x = self.mlp(x)
        x = x + identity

        return x


class SequenceClassifier(nn.Module):

    def __init__(self):
        super().__init__()
            
        self.encoder = Encoder(d_model=768, num_heads=12, num_encoder_blocks=2)
        self.head = nn.Linear(in_features=768, out_features=3)

    def forward(self, x):
        x = self.encoder(x)
        x = x[:, -1, :]
        x = self.head(x)
        
        return x
        


In [22]:
class Embedder():

    def __init__(self):

        self.tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
        self.model = BertModel.from_pretrained("bert-base-uncased")

        for param in self.model.parameters():
            param.requires_grad = False
    
    def __call__(self, sentences: str):

        if isinstance(sentences, torch.Tensor):
            sentences = list(sentences)
            
        input_ids = self.tokenizer(sentences, padding=True, truncation=True, return_tensors="pt")["input_ids"]
        embeddings = self.model.embeddings(input_ids)

        return embeddings

In [23]:
class ReviewDataset(Dataset):

    def __init__(self, reviews, ratings):
        super().__init__()

        self.reviews = reviews
        self.ratings = ratings
        self.embedder = Embedder()

    def __len__(self):
        return len(self.reviews)
    
    def __getitem__(self, idx):
        return self.reviews[idx], self.ratings[idx]
    

In [24]:
model = SequenceClassifier().to("cuda")
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.001, weight_decay=0.001)
loss_fn = nn.CrossEntropyLoss()

RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

In [122]:
train_dataset = ReviewDataset(reviews=x_train, ratings=y_train)
test_dataset = ReviewDataset(reviews=x_test, ratings=y_test)

train_dataloader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=True)
test_dataloader = DataLoader(dataset=test_dataset, batch_size=32, shuffle=True)

In [123]:
torch.cuda.empty_cache()

In [ ]:
epochs = 20
embedder = Embedder()

for epoch in range(1, epochs+1):
    print("Starting Training...")

    running_train_loss = 0
    train_total = 0
    train_correct = 0

    model.train()
    for reviews, ratings in tqdm(train_dataloader):
        embeddings = embedder(reviews).to("cuda")
        ratings = ratings.to("cuda")

        optimizer.zero_grad()
        logits = model(embeddings)

        loss = loss_fn(logits, ratings)
        loss.backward()
        optimizer.step()

        running_train_loss += loss.item()
        train_total += len(reviews)
        prediction = torch.argmax(logits, dim=1)
        train_correct += (prediction == ratings).sum().item()

    print(f"Avg loss: {running_train_loss / train_total}, accuracy: {train_correct / train_total}")

    running_eval_loss = 0
    eval_total = 0
    eval_correct = 0

    model.eval()
    with torch.no_grad():
        for reviews, ratings in tqdm(test_dataloader):
            embeddings = embedder(reviews).to("cuda")
            ratings = ratings.to("cuda")

            logits = model(embeddings)
            loss = loss_fn(logits, ratings)

            running_eval_loss += loss.item()
            eval_total += len(reviews)
            predicted = torch.argmax(logits, dim=1)
            eval_correct += (predicted == ratings).sum().item()

    print(f"Avg loss: {running_eval_loss / eval_total}, accuracy: {eval_correct / eval_total}")



Starting Training...


100%|██████████| 835/835 [01:21<00:00, 10.30it/s]


Avg loss: 0.03392066523100306, accuracy: 0.583021410390777


100%|██████████| 209/209 [00:09<00:00, 22.63it/s]


Avg loss: 0.03165995234143948, accuracy: 0.6161676646706586
Starting Training...


...

Avg loss: 0.02820904734516572, accuracy: 0.649251497005988
Starting Training...


100%|██████████| 835/835 [01:17<00:00, 10.74it/s]


Avg loss: 0.02712600261892168, accuracy: 0.6563482557269052


100%|██████████| 209/209 [00:08<00:00, 23.35it/s]

Avg loss: 0.0274582736238748, accuracy: 0.6546407185628742
